# Initialising Your OpenMined Duet Authority Agent as an Issuer

## The OM Authority actor is **ONLY** included to allow you to complete this example without relying on an externally hosted service. The happy path for completing this tutorial however is to interact with a user interface for a Full Stack Application acting as the OM Authority. It just depends if this gets completed, hosted and remains online at the time you are taking this course.

#### The code for the Full Stack Application can be found here: https://github.com/wip-abramson/fpc-om-authority-aries-application

#### The Full Stack Application should be running here: <some location to be confirmed>
    
If the application is not accessible you can either attempt to run the Full Stack App from the code linked above. Or you can run through these notebooks to issue the Data Scientist and Data Owner the relevant credentials.
    
These notebooks are designed to replicate much of the logic used in the Full Stack Application.
    
#### Also Note: If running the OM Authority through these notebooks then you should only need to run this notebook once. Use `./manage.sh start` to stop the containers and the state will be persisted


### Imports

In [ ]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [ ]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(
    f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}"
)
agent_controller = AriesAgentController(admin_url, api_key)

## Write DID to the Public Ledger

Note: if defined a ACAPY_WALLET_SEED value for your agent then this function will return a DID, but this DID still needs to be written to the ledger. If you did not define a seed you will need to create a DID first.

In [ ]:
public_did_response = await agent_controller.wallet.get_public_did()

In [ ]:
if public_did_response["result"]:
    did_obj = public_did_response["result"]
else:
    create_did_response = await agent_controller.wallet.create_did()
    did_obj = create_did_response["result"]
print("DID", did_obj)

In [ ]:
# write new DID to Sovrin Stagingnet
import requests
import json

url = "https://selfserve.sovrin.org/nym"

payload = {
    "network": "stagingnet",
    "did": did_obj["did"],
    "verkey": did_obj["verkey"],
    "paymentaddr": "",
}

# Adding empty header as parameters are being sent in payload
headers = {}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.json())

## Accept Transaction Author Agreement

Although the Sovrin StagingNet is permissionless, before DID's have the authority to write to the ledger they must accept something called a transaction author agreement by signing it using the DID they have on the ledger.

As a global public ledger, the Sovrin Ledger and all its participants are subject to privacy and data protection regulations such as the EU General Data Protection Regulation (GDPR). These regulations require that the participants be explicit about responsibilities for Personal Data.

To clarify these responsibilities and provide protection for all parties, the Sovrin Governance Framework Working Group developed an agreement between Transaction Authors and the Sovrin Foundation. The TAA can be found at [Sovrin.org](https://Sovrin.org). It ensures that users are aware of and consent to the fact that all data written to the Sovrin Ledger cannot be removed, even if the original author of the transaction requests its removal.

The TAA outlines the policies that users must follow when interacting with the Sovrin Ledger. When a user’s client software is preparing a transaction for submission to the network, it must include a demonstration that the user had the opportunity to review the current TAA and accept it. This is done by including some additional fields in the ledger write transaction: 

* A hash of the agreement
* A date when the agreement was accepted
* A string indicating the user interaction that was followed to obtain the acceptance

The Indy client API used by Sovrin has been extended to allow users to review current and past agreements and to indicate acceptance through an approved user interaction pattern. - source: https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/

For more details on TAA please read more at the following links:
* [Preparing for the Sovrin Transaction Author Agreement](https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/)
* [How the recent approval of the Sovrin Governance Framework v2 affects Transaction Authors
](https://sovrin.org/how-the-recent-approval-of-the-sovrin-governance-framework-v2-affects-transaction-authors/)
* [TAA v2](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/TAA.md)
* [TAA Acceptance Mechanism List (AML)](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/AML.md)

In [ ]:
taa_response = await agent_controller.ledger.get_taa()
TAA = taa_response["result"]["taa_record"]
TAA["mechanism"] = "service_agreement"
await agent_controller.ledger.accept_taa(TAA)

## Assign Agent Public DID if Not Set

Will only be ran if ACAPY_WALLET_SEED not initially set.

In [ ]:
if did_obj["posture"] != "public":
    response = await agent_controller.wallet.assign_public_did(did_obj["did"])
print("Successfully intialised agent with Public DID : ", did_obj["did"])

## Identifying Schema to Use

These are the same schema used for the previous set of notebooks in the public key infrastructures course

In [ ]:
# Copy these for the next notebook 👇🏾

data_scientist_schema_id = "Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Scientist:0.0.1"
data_owner_schema_id = "Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1"

## Writing Credential Definitions

For all schema you intend to issue credentials against your agent must author a credential definition transaction to the public ledger. This specifies the public cryptographic material your agent will use to sign all credentials issued against a specific schema. 

In [ ]:
cred_def_response = await agent_controller.definitions.write_cred_def(
    data_scientist_schema_id
)
data_scientist_cred_def_id = cred_def_response["credential_definition_id"]

In [ ]:
cred_def_response = await agent_controller.definitions.write_cred_def(
    data_owner_schema_id
)
data_owner_cred_def_id = cred_def_response["credential_definition_id"]

## Copy Across Credential Definitions to the om_authority_issue.ipynb file


In [ ]:
print("Copy these for the next notebook 👇🏾\n")

print(f'data_scientist_cred_def_id = "${data_scientist_cred_def_id}"')
print(f'data_owner_cred_def_id = "${data_owner_cred_def_id}"')

## Terminate Controller


In [ ]:
await agent_controller.terminate()